In [1]:
import bs4  
from bs4 import BeautifulSoup  
import pandas as pd
import scipy as sc
import numpy as np
import requests

import selenium
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

In [2]:
def load_object(url_address):
    src=requests.get(url_address) 
    #src.content - to make a request to get the content of the webpage
    return BeautifulSoup(src.content, "lxml")

We use selenium to load the HTML code of the main WEB to a soup object:

In [3]:
my_url = 'https://www.ulta.com/' 
option = Options()
option.headless = False
driver = webdriver.Chrome(options=option)
driver.get(my_url)
driver.maximize_window()
action = webdriver.ActionChains(driver)
chart_search = WebDriverWait(driver,
                             20).until(EC.presence_of_element_located((By.CLASS_NAME, 'back-top')))

html = driver.page_source
driver.quit()
soup = BeautifulSoup(html)
temp = soup.find_all("div",{"class": "DesktopLeftNav__Header"})[1]
departments = temp.find_all("div",{"class": "DesktopLeftNav__navItemContainer"})



After we load the HTML code to a soup object, we are crowling with BeautifulSoup library

In [4]:
Url_List=[]  #list of all the linksite of the categories in all departments
Categories_types=[]
main_Categories=[]

for a in departments:
    linksite= a.find('a', {"class":"Anchor"})['href']    
    
    MainPage_Soup=load_object(linksite)
    categories=MainPage_Soup.find("li",attrs={"class":"cat-sub-nav"}).findAll("li") if MainPage_Soup.find("li",attrs={"class":"cat-sub-nav"}) else None
    
    if(categories):  
        for category in categories: #in every department there is categorys (and in every category there is products)
            
            tmp=a.find('a', {"class":"Anchor"}).get_text().strip() #tmp==main category name 
            if tmp=='Gifts' or tmp=='Tools & Brushes': continue
                
            main_Categories.append(tmp)
            Categories_types.append(category.find("a").get_text().strip())   
            
            url_address="https:"+category.find("a").attrs['href']
            Url_List.append(url_address)

In [5]:
mainCategories=[]
Categories=[]
Name=[]
Brand_name=[]
Rating=[]
Price=[]
Links=[]
    
for url_address in Url_List: #in evey Main category there is a few categories        
    for page in range(15):  #in evey category there is a few pages 
           
        soup=load_object(url_address)
        objects=soup.findAll("div",attrs={"class":"productQvContainer"}) 
          
        for obj in objects: #עבור כול מוצר בדף
            
            mainCategories.append(main_Categories[0])
            Categories.append(Categories_types[0])
            
            Brand_name.append(obj.find("div",attrs={"class":"prod-title-desc"}).find("a").string[5:])
            Name.append(obj.find("div",attrs={"class":"prod-title-desc"}).find("p").find("a").string[5:])
            Rating.append(obj.find("label",attrs={"class":"sr-only"}).string[:3] if obj.find("label",attrs={"class":"sr-only"}) else None)
            Price.append(obj.find("div",attrs={"class":"productPrice"}).get_text()[14:30].strip() if obj.find("div",attrs={"class":"productPrice"}) else None)
                  
            link_for_product=obj.find("div",attrs={"class":"quick-view-prod"}).find("a").attrs['href']
            if link_for_product[0:5]!="https":
                link_for_product="https://www.ulta.com"+link_for_product 
            Links.append(link_for_product)
                
        try:
            url_address="https://www.ulta.com/"+soup.select_one('.next-prev .next').attrs['href']
        except:
            break
            
    main_Categories.pop(0)
    Categories_types.pop(0)
                
dataSet=pd.DataFrame(data={'Main Category':mainCategories,'Category':Categories,'Product name':Name,'Brand':Brand_name,'Rating':Rating,'Price':Price,'Link':Links})

In [6]:
dataSet

,Main Category,Category,Product name,Brand,Rating,Price,Link
0,Makeup,Face,Kylighter,KYLIE COSMETICS,3.6,$8.00,https://www.ulta.com/p/kylighter-pimprod2007188
1,Makeup,Face,Shape Tape Concealer,Tarte,4.5,$29.00,https://www.ulta.com/p/shape-tape-concealer-xl...
2,Makeup,Face,Skin Concealer,KYLIE COSMETICS,3.7,$5.00,https://www.ulta.com/p/skin-concealer-pimprod2...
3,Makeup,Face,CC+ Cream with SPF 50+,It Cosmetics,4.3,$39.50,https://www.ulta.com/p/cc-cream-with-spf-50-xl...
4,Makeup,Face,Double Wear Stay-in-Place Makeup,Estée Lauder,4.6,$43.00,https://www.ulta.com/p/double-wear-stay-in-pla...
...,...,...,...,...,...,...,...
21316,Men,Gifts & Value Sets,Wanted Eau de Toilette Set,Azzaro,Pri,$122.00,https://www.ulta.com/p/wanted-eau-de-toilette-...
21317,Men,Gifts & Value Sets,Grooming Essentials Kit,STMNT Grooming Goods,Pri,$49.95,https://www.ulta.com/p/grooming-essentials-kit...
21318,Men,Gifts & Value Sets,Wanted By Night Set,Azzaro,Pri,$97.00,https://www.ulta.com/p/wanted-by-night-set-pim...
21319,Men,Gifts & Value Sets,Legendary Grooming Kit,MASON MAN,5.0,$21.00,https://www.ulta.com/p/legendary-grooming-kit-...


We change the string value of 'Rating' column in to numric value:

In [7]:
dataSet['Rating'] = pd.to_numeric(dataSet['Rating'],errors='coerce')

Cleaning data:

In [8]:
dataSet=dataSet.drop_duplicates(subset=['Link']) #products with the same linksite point to the Same product 
dataSet=dataSet.dropna() #deleting product with NULL columns
dataSet=dataSet.reset_index(drop=True)

In [9]:
dataSet.to_csv('dataSet.csv')

In [10]:
dataSet

,Main Category,Category,Product name,Brand,Rating,Price,Link
0,Makeup,Face,Kylighter,KYLIE COSMETICS,3.6,$8.00,https://www.ulta.com/p/kylighter-pimprod2007188
1,Makeup,Face,Shape Tape Concealer,Tarte,4.5,$29.00,https://www.ulta.com/p/shape-tape-concealer-xl...
2,Makeup,Face,Skin Concealer,KYLIE COSMETICS,3.7,$5.00,https://www.ulta.com/p/skin-concealer-pimprod2...
3,Makeup,Face,CC+ Cream with SPF 50+,It Cosmetics,4.3,$39.50,https://www.ulta.com/p/cc-cream-with-spf-50-xl...
4,Makeup,Face,Double Wear Stay-in-Place Makeup,Estée Lauder,4.6,$43.00,https://www.ulta.com/p/double-wear-stay-in-pla...
...,...,...,...,...,...,...,...
15569,Men,Hair,Cedarwood Grooming Paste,Every Man Jack,0.0,$9.99,https://www.ulta.com/p/cedarwood-grooming-past...
15570,Men,Hair,Daily Hydrator Hair Styling Cream,Frederick Benjamin,5.0,$12.00,https://www.ulta.com/p/daily-hydrator-hair-sty...
15571,Men,Gifts & Value Sets,Wicked Beard Trio,Billy Jealousy,4.5,$14.4,https://www.ulta.com/p/wicked-beard-trio-xlsIm...
15572,Men,Gifts & Value Sets,Deluxe Grooming Kit,American Crew,0.0,$23.50,https://www.ulta.com/p/deluxe-grooming-kit-pim...
